<a href="https://colab.research.google.com/github/Hedredo/dagshub_p7/blob/main/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **How-To MLflow**

**How to use MLflow to log params and metrics**

```
with mlflow.start_run():
  # Your training code here...
  mlflow.log_metric('accuracy', 42)
  mlflow.log_param('Param name', 'Value')
```


**Turn on autologging for most popular ML frameworks**<br>
For more info and list of supported frameworks, see: https://mlflow.org/docs/latest/tracking.html#automatic-logging<br>



```
mlflow.autolog()
```



# **Configuration de l'environnement**

In [1]:
%pip install -q dagshub mlflow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.9/247.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.6 MB/s eta 0:00:00
   ━

In [2]:
# Check the CUDA version with the T4 GPU instance
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
import dagshub
import mlflow
import warnings
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import spacy
from google.colab import drive

from sklearn.model_selection import train_test_split

In [4]:
print("Tensorflow framework: GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Pytorch framework: GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")

Tensorflow framework: GPU is available
Pytorch framework: GPU is available


In [5]:
# Remove FutureWarning alerts
warnings.filterwarnings('ignore', category=FutureWarning)

# Initialize dagshub repo
dagshub.init(repo_owner='hedredo', repo_name='dagshub_p7', mlflow=True)

# Set a random seed
SEED = 314
np.random.seed(SEED)
print("Random seed set to", SEED)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=815fd434-bd5c-4e39-b248-3384ce77b4a1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=7acf2878dcc02b42cf6e8cb4f3e11ec1b33da8d0fdbf5684b564cca34cf618a4




Accessing as hedredo

Initialized MLflow to track repo "hedredo/dagshub_p7"

Repository hedredo/dagshub_p7 initialized!

Random seed set to 314


In [6]:
# Mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Download the small language model for spacy
spacy.cli.download('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Chargement des données

In [8]:
# Path to the csv file
path = "/content/drive/MyDrive/Formation & Emploi/OpenClassRooms/OC Projets de formation/Projet 7/training.1600000.processed.noemoticon.csv"

In [9]:
# Read the file
df = pd.read_csv(
    path,
    header=None,
    names=['target', 'ids', 'date', 'flag', 'user', 'text'],
    parse_dates=['date'],
    encoding='utf-8',
    encoding_errors='replace' # replace the errors with unicode symbol � (U+FFFD)
    )

# Assign the len of the original file
df_len = len(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   target  1600000 non-null  int64         
 1   ids     1600000 non-null  int64         
 2   date    1600000 non-null  datetime64[ns]
 3   flag    1600000 non-null  object        
 4   user    1600000 non-null  object        
 5   text    1600000 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 73.2+ MB


- target : cible composée de 0:negative et 4:positive uniforme (environ 50 50)
- id : identifiant unique pour chaque tweet. Un tweet peut être en double avec un même id annoté soit positif (4) soit négatif (0). C'est le seul champs qui peut varier.
- date : la date du tweet (l'heure n'est pas fournie). PDT est l'heure du pacifique. Tous les tweets sont en PDT et la valeur est retiré lors du parsing
- flag : toutes les valeurs sont à no query, useless
- user : nom de l'utilisateur qui a posté le tweet. Peut être présent sur plusieurs tweets.
- text : le texte du tweet

**BLOC-NOTES**
- Lire la description du projet
- Regarder les notes google drive de PA
- Faire un tableau markdown pour synthétiser le contenu
- Importer uniquement les données utiles
- Convertir les données dans le bon format
- Faire une EDA et voir par exemple si le jour, l'heure, le mois ont un impact sur le sentiment
- Faire une wordcloud pour voir les mots les plus fréquents selon la target
- Faire un comptage de mots, ponctuation, hashtag, mention, url et regarder si leur nombre a un impact sur la classe
- Préparer les corpus avec différentes options de prétraitement

# **Séparation des données**

In [11]:
# Define if working with a sample
sampling = True
proportion = 0.1

# Split the data with sampling or not
if sampling:
    if len(df) != df_len:
        print("Dataframe has already been sampled")
    else:
        print(f"Sampling the data with {proportion} fraction")
        df = df.sample(frac=proportion, random_state=SEED)

# Define X and y
X = df['text']
y = df['target']

# Split the data with a 0.2 test size
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=SEED
    )

# Display shape of splits
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Sampling the data with 0.1 fraction
X_train shape: (128000,)
X_test shape: (32000,)
y_train shape: (128000,)
y_test shape: (32000,)


# **Options de standardisation des textes**

Voici une liste de 10 moyens de standardiser votre corpus de tweets pour l'analyse de sentiments :

1. **Conversion en minuscules** : Transformer tous les caractères en minuscules pour uniformiser le texte.
2. **Suppression de la ponctuation** : Enlever tous les signes de ponctuation qui peuvent ne pas être pertinents pour l'analyse.
3. **Élimination des mots vides (stop words)** : Supprimer les mots courants qui n'ajoutent pas de valeur sémantique significative (comme "le", "la", "de", etc.).
4. **Stemming** : Réduire les mots à leur racine pour traiter les variations morphologiques (par exemple, "aimant", "aimer", "aimé" deviennent "aim").
5. **Lemmatisation** : Transformer les mots en leur forme de base ou canonique (par exemple, "étudiants" devient "étudiant").
6. **Suppression des nombres** : Enlever les chiffres qui peuvent ne pas être utiles pour l'analyse de sentiments.
7. **Traitement des URL** : Remplacer les liens par un token spécifique comme `<URL>` pour uniformiser le texte.
8. **Gestion des mentions** : Remplacer les @utilisateur par un token comme `<MENTION>` pour éviter les informations spécifiques inutiles.
9. **Traitement des hashtags** : Extraire le mot-clé du hashtag (par exemple, "#bonheur" devient "bonheur") pour conserver le sens.
10. **Gestion des emojis et émoticônes** : Remplacer les emojis par leur signification textuelle ou les supprimer si nécessaire (par exemple, "😊" devient "heureux").

Ces standardisations peuvent vous aider à améliorer la qualité de vos données et potentiellement les performances de vos modèles de classification en réduisant le bruit et en uniformisant le corpus.

In [14]:
# Load the en_core_web_sm model
spacy.prefer_gpu(gpu_id=0)
nlp = spacy.load('en_core_web_sm', disable=['ner'])